In [ ]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.backend import set_session

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)

### Create model

In [ ]:
keras.backend.set_learning_phase(0)
input_width = 251

backbone = keras.applications.MobileNetV2(
    input_shape=(128, input_width, 1),
    alpha=1.0,
    include_top=False,
    weights=None,
)
for layer in backbone.layers:
    if type(layer) == type(keras.layers.BatchNormalization()):
        layer.momentum = 0.9

backbone = keras.Model(inputs=backbone.input, outputs=backbone.get_layer("block_16_project_BN").output)
backbone.trainable = False
backbone.summary()

model = keras.Sequential([
    backbone,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(32),
    keras.layers.BatchNormalization(momentum=0.9),
    keras.layers.ReLU(),
    keras.layers.Dense(1, activation='sigmoid', name='fart_predict'),
])
model.summary()


### Load weights

In [ ]:
model.load_weights("./models_mnv2_1/finetune_checkpoint_mobile_2.hdf5")

### Convert to TFLite

In [ ]:
bin_path = '/home/david/anaconda3/envs/fart_detector_37/bin/'

if 'PATH' in os.environ:
    os.environ['PATH'] += ':' + bin_path
else:
    os.environ['PATH'] = bin_path

In [ ]:
sess = tf.keras.backend.get_session()
converter = tf.lite.TFLiteConverter.from_session(
    sess,
    model.inputs,
    model.outputs,
)
tflite = converter.convert()

with open('./models_mnv2_1/fart_detector.tflite', 'wb') as f:
    f.write(tflite)